In [10]:
!pip install PyJWT==1.7.1

  Using cached https://files.pythonhosted.org/packages/87/8b/6a9f14b5f781697e51259d81657e6048fd31a113229cf346880bb7545565/PyJWT-1.7.1-py2.py3-none-any.whl
ERROR: ibm-cloud-sdk-core 3.10.0 has requirement PyJWT<3.0.0,>=2.0.1, but you'll have pyjwt 1.7.1 which is incompatible.
  Found existing installation: PyJWT 2.1.0
    Uninstalling PyJWT-2.1.0:
      Successfully uninstalled PyJWT-2.1.0


In [11]:
!pip install ibm_watson bs4

  Using cached https://files.pythonhosted.org/packages/3f/32/d5d3cab27fee7f6b22d7cd7507547ae45d52e26030fa77d1f83d0526c6e5/PyJWT-2.1.0-py3-none-any.whl
  Found existing installation: PyJWT 1.7.1
    Uninstalling PyJWT-1.7.1:
      Successfully uninstalled PyJWT-1.7.1


In [12]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [13]:
recognition_service = SpeechToTextV1(IAMAuthenticator('xOht0gypqhEcJedUWdz8OV4V8xnFvNgNIEYQ_SOsthd3'))
recognition_service.set_service_url('https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/6e26e7ab-8373-4f28-aee8-2bda4f4da38c')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]


In [14]:
assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('odLgOEjhvCNZp9oyg9R2lWsJh-iJC63ut35g9CMTK_IL'))
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/f72e4303-ed83-4bc4-bbb2-9f6900f8e619')
ASSISTANT_ID = "34b711d7-5312-4920-846f-3e38933cd4df"
session_id = assistant.create_session(assistant_id =ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [17]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('GiV9UG6r9PLGIbKP36w2QM1crfTZg2KffqZY2V0zRV8d'))
synthesis_service.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/826f48e7-2211-40c1-b353-2b8e339ca189')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [18]:
speak_text(message_assistant(recognize_audio()))